In [175]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar
from pandas_profiling import ProfileReport
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
#import warnings; 
#warnings.simplefilter('ignore')
import seaborn as sns
import numpy as np

%matplotlib inline
plt.rcParams.update({'font.size': 16, 'figure.figsize': (16,6)})



In [50]:
import random

filename = 'data/training_set_VU_DM.csv'
p = 0.1  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)

#df = pd.read_csv(filename, nrows=1000)
df['date_time'] = pd.to_datetime(df['date_time'])

df_test = pd.read_csv('data/test_set_VU_DM.csv', header=0, skiprows=lambda i: i>0 and random.random() > p)
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

In [9]:
df.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

## Order of the price per srch_id, srch_destination_id, prop_id

In [182]:
df.sort_values(['srch_id', 'price_usd'], inplace=True)
df['index_per_srch_id'] = df.groupby(['srch_id']).cumcount()
df.sort_index(inplace=True)

In [183]:
df.sort_values(['srch_destination_id', 'price_usd'], inplace=True)
df['index_per_srch_destination_id'] = df.groupby(['srch_destination_id']).cumcount()
df.sort_index(inplace=True)

In [184]:
df.sort_values(['prop_id', 'price_usd'], inplace=True)
df['index_per_prop_id'] = df.groupby(['prop_id']).cumcount()
df.sort_index(inplace=True)

## Calculate date of trip
from: date_time + srch_booking_window\
until: date_time + srch_booking_window + srch_length_of_stay

In [176]:
df['date'] = df['date_time'].dt.floor('D')
df['days_until_trip'] = pd.to_timedelta(df['srch_booking_window'], unit='D')

df['start_date'] = df['date'] + df['days_until_trip']
df['end_date'] = df['date'] + df['days_until_trip'] + pd.to_timedelta(df['srch_length_of_stay'], unit='D')

## Count holidays (work free day) during the trip

In [178]:
cal = calendar()
holidays = cal.holidays(start=min(df['start_date']), end=max(df['end_date']))

df['holiday'] = 0
for holiday in holidays:
    df['holiday'] += (df['start_date'] <= holiday) & (df['end_date'] >= holiday)

## Try to separate business trips from holidays

sign for worktrip:\
srch_children_count = 0\
srch_adults_count = 1\
srch_saturday_night_bool = False\
srch_length_of_stay < avg\
orig_destination_distance < avg\
srch_booking_window < avg

In [173]:
df['business_trip'] = False
df.loc[(df['srch_children_count'] == 0) & (df['srch_adults_count'] == 1) & 
       (~df['srch_saturday_night_bool']) & (df['srch_length_of_stay'] <= 5), 'business_trip'] = True

## Some Hotels use price/day, other use price/trip
Try to find what price hotels showed and create features price_per_day and price_per_trip

In [166]:
# calculate difference between gross_bookings_usd and price_usd
# also calculate difference between gross_bookings_usd and srch_length_of_stay * price_usd

df['gross_price_diff'] = df['gross_bookings_usd'] - df['price_usd']
df['adj_price_diff'] = df['gross_bookings_usd'] - (df['srch_length_of_stay'] * df['price_usd'])

In [167]:
# price is given per day if srch_length_of_stay * price_usd is closer to gross_bookings_usd than
# price_usd

df['price_per_day_given'] = abs(df['adj_price_diff']) < abs(df['gross_price_diff'])

In [168]:
# only prop_id is correlated with price_per_day_given, other columns did not give good results
# so we find the prop_id that use price per day

df1 = df.groupby('prop_id')['price_per_day_given'].mean()
prop_id_with_price_per_day = df1[df1 > 0.5].index

In [169]:
# adjust price_per_day where price is given for whole trip
# and adjust price_per_trip where price is given per day

df['price_per_day'] = df['price_usd']
df['price_per_trip'] = df['price_usd']

df.loc[df['prop_id'].isin(prop_id_with_price_per_day), 'price_per_trip'] = df['price_per_day'] * df['srch_length_of_stay']
df.loc[~df['prop_id'].isin(prop_id_with_price_per_day), 'price_per_day'] = df['price_per_day'] / df['srch_length_of_stay']


In [170]:
# drop columns that helped creating price_per_day and price_per_trip but are not useful anymore

df.drop(['gross_price_diff', 'adj_price_diff', 'price_per_day_given'], axis=1, inplace=True)

## Average numeric features over:
srch_id\
visitor_location_country_id\
prop_id\
srch_destination_id\
prop_starrating

In [3]:
# add the average/min/max of these features over some column 
# for example: calc avg of these columns for every search_id
numeric_features = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_query_affinity_score', 'orig_destination_distance']

In [4]:
# groupy by these columns
groupby_columns = ['srch_id', 'visitor_location_country_id', 'prop_id', 'prop_starrating', 'srch_destination_id']
df_groupbys = []

# first create aggreagted all dataframes over specified columns, adjust column names
for split_col in groupby_columns:
    # aggregate mean
    df_groupby_mean = df.groupby(split_col)[numeric_features].mean()
    new_colnames = []
    for col in numeric_features:
        new_colnames.append('mean_' + col + '_over_' + split_col)
    df_groupby_mean.columns = new_colnames
    #df_groupbys.append(df_groupby_mean)
    
    # aggregate min
    df_groupby_min = df.groupby(split_col)[numeric_features].min()
    new_colnames = []
    for col in numeric_features:
        new_colnames.append('min_' + col + '_over_' + split_col)
    df_groupby_min.columns = new_colnames
    #df_groupbys.append(df_groupby_min)
    
    # aggregate max
    df_groupby_max = df.groupby(split_col)[numeric_features].max()
    new_colnames = []
    for col in numeric_features:
        new_colnames.append('max_' + col + '_over_' + split_col)
    df_groupby_max.columns = new_colnames
    #df_groupbys.append(df_groupby_max)
    df_groupbys.append(pd.concat([df_groupby_mean, df_groupby_min, df_groupby_max], axis=1))
    
# merge oridinal df with the aggregated dataframes
for col, df_groupby in zip(groupby_columns, df_groupbys):
    df = pd.merge(df, df_groupby, right_index=True, left_on = col, how='left')

In [5]:
# normed difference between value and mean/min/max aggregated values
# for example: (value - mean(aggregated values)) / mean(aggregated values)
for split_col in groupby_columns:
    for col1 in numeric_features:
        #diff to mean
        col2 = 'mean_' + col1 + '_over_' + split_col
        df['diff_to_' + col2] = (df[col1] - df[col2]) / df[col2]
        
        #diff to min
        col2 = 'min_' + col1 + '_over_' + split_col
        df['diff_to_' + col2] = (df[col1] - df[col2]) / df[col2]
        
        #diff to max
        col2 = 'max_' + col1 + '_over_' + split_col
        df['diff_to_' + col2] = (df[col1] - df[col2]) / df[col2]

/var/folders/qd/2v2_mp9x4vq3mdm35p6mbz6h0000gn/T/ipykernel_30319/1347481713.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['diff_to_' + col2] = (df[col1] - df[col2]) / df[col2]
/var/folders/qd/2v2_mp9x4vq3mdm35p6mbz6h0000gn/T/ipykernel_30319/1347481713.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['diff_to_' + col2] = (df[col1] - df[col2]) / df[col2]
/var/folders/qd/2v2_mp9x4vq3mdm35p6mbz6h0000gn/T/ipykernel_30319/1347481713.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

In [6]:
df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,diff_to_max_srch_children_count_over_srch_destination_id,diff_to_mean_srch_room_count_over_srch_destination_id,diff_to_min_srch_room_count_over_srch_destination_id,diff_to_max_srch_room_count_over_srch_destination_id,diff_to_mean_srch_query_affinity_score_over_srch_destination_id,diff_to_min_srch_query_affinity_score_over_srch_destination_id,diff_to_max_srch_query_affinity_score_over_srch_destination_id,diff_to_mean_orig_destination_distance_over_srch_destination_id,diff_to_min_orig_destination_distance_over_srch_destination_id,diff_to_max_orig_destination_distance_over_srch_destination_id
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,59526,3,3.5,...,-1.0,-0.108091,0.0,-0.875000,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,95307,4,3.5,...,-1.0,-0.108091,0.0,-0.875000,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,114766,2,3.5,...,-1.0,-0.108091,0.0,-0.875000,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,56063,4,4.5,...,-1.0,-0.117057,0.0,-0.750000,NaN,NaN,NaN,-0.889549,1082.318182,-0.974465
4,4,2012-12-31 08:59:22,5,219,NaN,NaN,219,64344,4,3.0,...,-1.0,-0.117057,0.0,-0.750000,NaN,NaN,NaN,-0.889516,1082.636364,-0.974457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495579,332781,2013-03-03 15:12:59,5,219,NaN,NaN,219,18007,3,3.5,...,-1.0,0.000000,0.0,0.000000,-0.0,-0.0,-0.0,-0.051054,1.284689,-0.453252
495580,332782,2012-11-08 18:24:21,19,158,NaN,NaN,158,26348,4,0.0,...,-1.0,-0.090909,0.0,-0.500000,NaN,NaN,NaN,NaN,NaN,NaN
495581,332782,2012-11-08 18:24:21,19,158,NaN,NaN,158,108579,4,4.5,...,-1.0,-0.090909,0.0,-0.500000,NaN,NaN,NaN,NaN,NaN,NaN
495582,332784,2013-06-18 14:35:24,24,219,NaN,NaN,100,12507,4,4.5,...,-1.0,-0.123468,0.0,-0.875000,NaN,NaN,NaN,-0.409728,2350.785714,-0.915389


In [ ]:
#TODO
#calculate diff between price_usd and gross_bookings_usd